# 다항회귀
> 선형회귀의 경우 설명변수와 종속변수의 관계를 선형관계로 해석하지만 현실 문제는 선형으로 해결이 불가능한 경우가 많습니다.  
선형모델의 구조적 한계를 보완하고자 **변수 간 영향력에 해당하는 새로운 변수를 생성**하여 방정식을 비선형으로 만들고 선형모델을 적용시킬 수 있습니다.  
2차원 예시 $x_1, x_2, x_3$를 $x_1^2, x_2^2, x_3^2, x_1x_2, x_1x_3, x_2x_3$으로 변환하여 변수를 확장.  
보통 2차원 또는 3차원까지 적용하며, 변수가 많아지므로 Ridge, Lasso 모델을 적용한다.
<img src="./image/47.png">

### 다항회귀 실습

In [ ]:
import pandas as pd

In [ ]:
# boston 데이터 확인
df = pd.read_csv('./data/boston.csv')

In [ ]:
# 타겟데이터 분할
y = df['y']
X = df.drop('y', axis=1)

In [ ]:
# 훈련셋과 테스트셋 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# 전처리 모델 import
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# 다항변수 제작함수 인스턴스화
poly = PolynomialFeatures(degree=2, include_bias=False)
'''
파라메터
degree=2 : 차수설정
include_bias=False : 상수항 제거(필수)
'''

In [ ]:
X_train

In [ ]:
# 변수 변환 학습 컬럼갯수와 컬럼명 정도를 학습. 과 동시에 원본값에 학습 결과를 적용
poly_X_train = poly.fit_transform(X_train)

In [ ]:
# shape 확인
poly_X_train.shape

In [ ]:
# 변환식 반환
poly.get_feature_names_out()

In [ ]:
# 표시 가능한 최대 컬럼 갯수 설정
pd.options.display.max_columns = 150
pd.options.display.max_rows = 150

In [ ]:
# 데이터프레임으로 제작 후 데이터 확인
pd.DataFrame(poly_X_train, columns=poly.get_feature_names_out())

In [ ]:
# test 데이터 동일한 모델로 적용
poly_X_test = poly.transform(X_test)

In [ ]:
# 모델 정의
from sklearn.linear_model import Lasso
poly_lasso = Lasso()

In [ ]:
# 모델 학습
poly_lasso.fit(poly_X_train, y_train)

In [ ]:
# 모델 예측
poly_lasso_pred = poly_lasso.predict(poly_X_test)

In [ ]:
# 모델 평가
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
print(f'R2 score : {r2_score(y_test, poly_lasso_pred)}')
print(f'RMSE : {mean_squared_error(y_test, poly_lasso_pred, squared=False)}') # squared=False 루트 적용한 값
print(f'MAE : {mean_absolute_error(y_test, poly_lasso_pred)}')

In [ ]:
'''
linear
R2 score : 0.7112260057484923
RMSE : 4.638689926172829
MAE : 3.1627098714574156
'''

In [ ]:
# 모델 계수 확인
pd.DataFrame(poly_lasso.coef_, index=poly.get_feature_names_out())

In [ ]:
# 계수 플로팅
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 20))
plt.barh(poly.get_feature_names_out(), poly_lasso.coef_)